In [1]:
# General:
import tweepy           # Para consumir la API de Tweeter
import pandas as pd     # Para análisis de datos
import numpy as np      # Para cálculo numérico

import networkx as nx
from pyvis.network import Network

import pickle
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

import tensorflow as tf

import os.path

import csv
import json

import datetime
import time


import ast


from gensim.utils import simple_preprocess ,tokenize
from gensim.models  import KeyedVectors

Using TensorFlow backend.


Funcion de llamada a la api

In [2]:
def twitter_setup():
    """
    Función de utilidad para configurar la API de Twitter
    con las claves de acceso.
    """
    
    # Autenticación y acceso usando claves:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    # Retornar API con autenticación:
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api

Funcion de extraccion de los tweets publicados en un dia 

In [3]:
def tweet_caller_csv(id_,star_year,start_month,star_day):
       
        extractor = twitter_setup() 
        u = extractor.get_user(id_)
        private= u.protected
        startdate = datetime.datetime(star_year,start_month,star_day)
        end_date= datetime.datetime(star_year,start_month,star_day+1)
        
        if os.path.isfile(u.screen_name + '_tweets.csv') == True:
            print('Archivo de tweets ya existe')
            pass
        else:
        
            if private == False:
                try: 

                        name = u.screen_name  
                        tweets = extractor.user_timeline(id=id_, count=200,tweet_mode= 'extended')
                        if len(tweets) > 0: 

                            alltweet = []
                            alltweet.extend(tweets)
                            oldest = alltweet[-1].id - 1

                            try:
                                while len(tweets) > 0:

                                        tweets = extractor.user_timeline(id= id_,count=200,tweet_mode= 'extended',max_id=oldest)
                                        alltweet.extend(tweets)
                                        oldest = alltweet[-1].id - 1
                            except ConnectionError:
                                print('ConnectionError')

                            result = [[tweet.id, tweet.created_at, tweet.full_text] for tweet in alltweet if tweet.created_at>startdate and tweet.created_at < end_date]
                            data= pd.DataFrame(result, columns=['id','date','text'])

                            data.to_csv( name + '_tweets.csv', line_terminator='\rn')
                        else:
                            data= pd.DataFrame(columns=['id','date','text'])

                            data.to_csv( name + '_tweets.csv', line_terminator='\rn')

                except Error as e:
                        print('Error en la creacion del archivo _tweets.csv del usuario:',id_)
                        print(e)
                        pass

            else:
                print('private user')
        return

Función de extraccion de seguidores de twitter

In [4]:
def extractor_follows(id_):
    extractor= twitter_setup()
    ids = []
    u = extractor.get_user(id_)
    name = u.screen_name
    
    try:
        for page in tweepy.Cursor(extractor.followers_ids, id=id_).pages():
            ids.extend(page)
    except: ConnectionError     
        
    ids=pd.DataFrame(ids,columns=[name],dtype=object)
    ids.to_csv(name + '_followers.csv',mode= 'a')
    
    return ids

Función de extraccion de seguidores con nombre

In [5]:
def extractor_follows_with_name(id_):
    extractor= twitter_setup()
    ids = []
    try:
        u = extractor.get_user(id_)
        name = u.screen_name
        try:
            for page in tweepy.Cursor(extractor.followers_ids, id=id_).pages():
                ids.extend(page)
            
        except:
                pass
            
        ids_dataframe=pd.DataFrame(ids,columns=[name],dtype=object)
        ids_dataframe.to_csv(name + '_followers.csv',mode= 'a')
        
    except tweepy.TweepError as e: 
        print('Usuario_borrado')
        
        pass
    
    
    return ids

Función de para guardar el diccionario de predicciones en un archivo

In [6]:
def save_dict_to_file(dic,name):
    f = open(name+'predict_dict.txt','w')
    f.write(str(dic))
    f.close()

Función de para abrir el diccionario de predicciones de un archivo

In [7]:
def load_dict_from_file(name):
    f = open(name+'predict_dict.txt','r')
    data=f.read()
    f.close()
    return ast.literal_eval(data)

In [8]:
def Average(lista):
    
    lista_limpia=[]
    for i in lista:
        if type(i) == str:
            pass 
        else:
            lista_limpia.append(i)
            
    
    
    if len(lista_limpia)== 0:
        lista= 'None'
    else:
        lista= sum(lista_limpia)/len(lista_limpia)
    
    return lista

In [9]:
def prediction(texto):
    global model_w2v
    global model 
    maxlen=30
    prediction_matrix=[]
    a = simple_preprocess(texto)
    prediction_matrix.append([ model_w2v.vocab[w].index for w in a if w in model_w2v.vocab])
    prediction_matrix= pad_sequences(prediction_matrix, padding='post', maxlen=maxlen)
    class_= model.predict_classes(prediction_matrix)
    probability= model.predict_proba(prediction_matrix)
    return int(class_)

In [10]:
def prediction_dict_first_level(id_,star_year,start_month,star_day):
    
    
    extractor= twitter_setup()
    
    
    try:
        u = extractor.get_user(id_)
        name = u.screen_name
        dt= pd.read_csv(name + '_followers.csv',index_col= 0)
        #Declarar el diccionario de predicciones 
        dicts= {}
        for i in dt[name]:
            try:
                user= extractor.get_user(i)
                
                if user.protected == False:
                    #extraer tweets y guardarlos en un csv
                    tweet_caller_csv(i,star_year,start_month,star_day)
                    #Cargar el archivo con los tweets
                    tweets= pd.read_csv(user.screen_name +'_tweets.csv',dtype= object)
                    #Obtener el texto de los tweets
                    tweets= tweets['text'].astype(str)

                    #Bucle de predicciones
                    predictions=[]
                    for v in tweets:
                        #introducir en un lista las predicciones de los tweets 
                        predictions.append(prediction(v))
                    #obtener la media de las predicciones 
                    predictions_mean= Average(predictions)

                    dicts[i]= predictions_mean

                else:
                    #el usuario es privado
                    dicts[i] = 'Private_user'
                    
            except tweepy.TweepError as e: 
                print('Usuario_borrado')  
        #Guardar el diccionario de predicciones en un archivo
        
        save_dict_to_file(dicts,name) 
   
    except tweepy.TweepError as e: 
         print('Usuario_borrado')  
        
     
    

In [11]:
def prediction_dicts_second_level(id_,star_year,start_month,star_day):
    
    extractor= twitter_setup()
    u = extractor.get_user(id_)
    name = u.screen_name
    dt= pd.read_csv(name + '_followers.csv',index_col= 0)
    
    for i in dt[name]:
        try:
            user= extractor.get_user(i)
            user_name= user.screen_name
            print(user_name)
            user_dt = pd.read_csv(user_name+'_followers.csv',index_col= 0)
            #Declarar el diccionario de predicciones 
            dicts= {}
            #Obtener el segundo nivel de usuarios
            for z in user_dt[user_name]:
                try:
                    user_follower= extractor.get_user(z)
                    print(user_follower.screen_name)
                    #Es posible que el seguidor ya este en un diccionario de predicciones
                    if os.path.isfile(user_follower.screen_name + '_followers.csv') == True:
                        pass
                    #Si no existe el archivo no estan realizadas las predicciones     
                    else:    

                        if user_follower.protected == False:
                            #Extraer tweets y guardarlos en un csv
                            tweet_caller_csv(z,star_year,start_month,star_day)
                            #Cargar el archivo con los tweets
                            tweets= pd.read_csv(user_follower.screen_name +'_tweets.csv',dtype= object)
                            #Obtener el texto de los tweets
                            tweets= tweets['text'].astype(str)

                            #Bucle de predicciones
                            predictions=[]
                            for v in tweets:
                                #Introducir en un lista las predicciones de los tweets 
                                predictions.append(prediction(v))
                            #Obtener la media de las predicciones 
                            predictions_mean= Average(predictions)

                            dicts[z]= predictions_mean

                        else:
                            #El usuario es privado
                            print('usuario privado')
                            dicts[z] = 'Private_user'
                            
                except tweepy.TweepError as e: 
                    print('Usuario_borrado')
               
            #Guardar el diccionario de predicciones en un archivo
            save_dict_to_file(dicts,user_name)
            
        except tweepy.TweepError as e: 
                print('Usuario_borrado')                



In [12]:
def sentiment_graph_constructor(id_,star_year,start_month,star_day):
    extractor= twitter_setup()
    u = extractor.get_user(id_)
    name = u.screen_name
    #cargar los nodos (ids unicos de los followers)
    dt = pd.read_csv(name + '_followers.csv',index_col= 0)
    
    list_of_followers =[]
    #extraemos todos los ids de los seguidores
    for i in dt[name]:
        if i == id_:
            pass
        else:
            list_of_followers.append(i)
            user = extractor.get_user(i)
            df = pd.read_csv(user.screen_name + '_followers.csv',index_col=0)
            for v in df[user.screen_name]:
                if v == id_:
                    pass
                else:
                    list_of_followers.append(str(v))
    
    #transformamos la lista de seguidores en una lista con valores unicos
    nodes= unique(list_of_followers)
    # borramos la lista de seguidores para ahorrar espacion en memoria
    del list_of_followers
    
    #Declaramos la grafica y añadimos el primer nodo en color verde para identificarlo
    g= Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    
    g.add_node(str(id_),label=name,color='#00ff1e')
    
    #Aplicamos las etiquetas de color en funcion de las predicciones para cada diccionario
    dict1 = color_labels(name)
    for i in dict1:
        g.add_node(str(i),lable= i,color=dict1.get(i))
 
    #Añadimos los seguidores de segundo nivel
    for i in dt[name]:
        user = extractor.get_user(i)
        dicts = color_labels(user.screen_name)
        for v in dicts:
            g.add_node(str(v),lable= v,color=dicts.get(v))
    
    
  
    #Construimos los ejes 
    edges = []
    new_nodes = []
    for i in dt[name]:
        if i == id_:
            pass
        else:
            #añadimos los ejes del usuario principal
            edges.append((str(id_),str(i)))
            #añadimos los ejes de los seguidores 
            user = extractor.get_user(i)
            df = pd.read_csv(user.screen_name + '_followers.csv',index_col=0)
            for v in df[user.screen_name]:
                if i == id_:
                    pass
                else:
                    edges.append((str(i),str(v)))
                    new_nodes.append(str(v))
    
    for y in new_nodes:
        g.add_node(str(y),label=y, color= "yellow" )
    
    not_nodes = []
    for i in nodes:
        if i not in g.nodes:
            not_nodes.append(i)
    
    edges_1=[]
    for i in edges:
        if i in not_nodes:
            pass
        else:
            edges_1.append(tuple(i))
            
    
    g.add_edges(edges_1)
    
    
    
    g.show(name+'_sentiment_graph.html')
    

In [13]:
def color_labels(name):
    #Carga el diccionario de predicciones del usuario dado
    dicts = load_dict_from_file(name)
    #Transformamos las predicciones realizadas en etiquitas de color
    
    print(type(dicts))
    for i in dicts:
        print(i)
        if type(i)==str:
            pass
        else:
            if type(dicts.get(i))== float:
                if dicts.get(i) >0.2:
                    dicts[i]= 'blue'
                else:
                    dicts[i]= 'red' 
            else:
                if dicts.get(i)== 'None':
                    dicts[i]= 'yellow'
                elif dicts.get(i) == 'Private_user':
                    dicts[i]= 'yellow'
                else:
                    pass #usuarios privados


    return dicts
                

In [14]:
def unique(list1): 
  
    
    unique_list = [] 
        
    for x in list1: 
        
        if x not in unique_list and x!= 0: 
            unique_list.append(x) 
    
    return unique_list

In [15]:
consumer_key='CTXchN99VNe7jl8MgZrE47AIm'
consumer_secret= 'faQ0b0r6FyfblBIPfJe0k6qjlEEqbynVmghjaKuzZhjskggjFq'
access_token = '1118494816105574401-JTGPAQhrSsvFSrd9O0eN03HkFVYjAK'
access_secret = 'OWg6aPlumQPSlXFn2kAiQcByNFkWyse0nE9JgXxQlglnR'

In [16]:
model_w2v = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.bin.gz', binary=True)
model = pickle.load(open('Keras_model_with_stop_words_0_67acc.sav', 'rb'))

C:\Users\Usuario\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [17]:
def extractor_all_followers(id_):
    followers = extractor_follows_with_name(id_)
    for i in followers:
        extractor = twitter_setup() 
        u = extractor.get_user(i)
        if os.path.isfile(u.screen_name + '_followers.csv') == True:
            print('El archivo ' + u.screen_name + '_followers.cvs ya existe ') 
            pass
        #Si no existe el archivo no estan realizadas las predicciones     
        else:  
            extractor_follows_with_name(i)
    return

In [18]:
def sentiment_network_twitter(id_,star_year,start_month,star_day):
    extractor = twitter_setup() 
    
    u = extractor.get_user(id_)
    
    if os.path.isfile(u.screen_name + '_followers.csv') == True:
        print('Archivo encontrado iniciando prediction_dict_first_level')
        prediction_dict_first_level(id_,star_year,start_month,star_day)
        print('********************** Iniciando prediction_dict_second_level*****************')
        prediction_dicts_second_level(id_,star_year,start_month,star_day)
        print('-----------------------construyendo grafo-----------------------------')
        sentiment_graph_constructor(id_,star_year,start_month,star_day)
        
    else:
        print('Archivo no encontrado, recolectando followers')
        extractor_all_followers(id_)
        prediction_dict_first_level(id_,star_year,start_month,star_day)
        prediction_dicts_second_level(id_,star_year,start_month,star_day)
        sentiment_graph_constructor(id_,star_year,start_month,star_day)
    return

In [19]:
%%time 
sentiment_network_twitter(924114887722364929,2020,5,5)

Archivo encontrado iniciando prediction_dict_first_level
Archivo de tweets ya existe
Archivo de tweets ya existe
Archivo de tweets ya existe
Archivo de tweets ya existe
Archivo de tweets ya existe
Archivo de tweets ya existe
Archivo de tweets ya existe
********************** Iniciando prediction_dict_second_level*****************
cotorres76
LuisGas98913305
Archivo de tweets ya existe
RNserUPCN
Archivo de tweets ya existe
rhlriver1976
Archivo de tweets ya existe
AlexanderCamejo
Archivo de tweets ya existe
marcos47614819
NormiMdq
usuario privado
alfredoeagustin
Archivo de tweets ya existe
Apolinario_Mam
Archivo de tweets ya existe
Carlos64Vega
Archivo de tweets ya existe
vientosi
Archivo de tweets ya existe
jlgpinasco
Archivo de tweets ya existe
danilosimian
Archivo de tweets ya existe
carlito60322427
Archivo de tweets ya existe
cobra_monica
Archivo de tweets ya existe
VickySalum
Archivo de tweets ya existe
datalinus
Archivo de tweets ya existe
Miguel_RomSa
Archivo de tweets ya existe
so

macaren07805122
Archivo de tweets ya existe
VapeoA
Archivo de tweets ya existe
AilinHuilliqueo
Archivo de tweets ya existe
matuastoblanco
Archivo de tweets ya existe
negrom17
Archivo de tweets ya existe
EscalanteMale
Archivo de tweets ya existe
chepasierra
Archivo de tweets ya existe
veroangel23
usuario privado
solange_erbin
usuario privado
RRelmuan
usuario privado
ChakamilLeon
Archivo de tweets ya existe
LRluisrojas
Archivo de tweets ya existe
CTAACapital
Archivo de tweets ya existe
Fede85409881
Archivo de tweets ya existe
huilliqueo_ivan
Archivo de tweets ya existe
JUANCASTROATE
Archivo de tweets ya existe
AteAbdala
Archivo de tweets ya existe
jorgeluisakatc1
Archivo de tweets ya existe
Sebasti45294120
Archivo de tweets ya existe
ChevesAle
Archivo de tweets ya existe
PalomaUriz
Archivo de tweets ya existe
ezequieljuan105
Archivo de tweets ya existe
Nahiir76164480
usuario privado
Maleescalante2
Archivo de tweets ya existe
pagliero_susana
Archivo de tweets ya existe
JigsawRoca
Archivo 

Archivo de tweets ya existe
nicolasveliz
Archivo de tweets ya existe
LibertadXD1
Archivo de tweets ya existe
KumpaLili
Archivo de tweets ya existe


Rate limit reached. Sleeping for: 325


varelacl
Archivo de tweets ya existe
esole_gonzalez
Archivo de tweets ya existe
MarceloPascu
Archivo de tweets ya existe
MargaWirkierman
Archivo de tweets ya existe
EdgardoRovira
Archivo de tweets ya existe
BravaLaPaula
Archivo de tweets ya existe
MilagrosTerra_
Archivo de tweets ya existe
JordiAguiar
Archivo de tweets ya existe
AlirioUribeMuoz
Archivo de tweets ya existe
JUNTOSXQUILMES
Archivo de tweets ya existe
CorredorCordi
laplataya
Archivo de tweets ya existe
MSequeyra
Archivo de tweets ya existe
elcornerdelsur
Archivo de tweets ya existe
hedyrodriguez27
usuario privado
Joselodeza
Archivo de tweets ya existe
adrian_ttc
Archivo de tweets ya existe
Carinalopezm
Archivo de tweets ya existe
GMTVcanal34
Archivo de tweets ya existe
Descacomuna1
Archivo de tweets ya existe
MineralsF
Archivo de tweets ya existe
gatolecat
Archivo de tweets ya existe
Rous11867938
Archivo de tweets ya existe
PescadoDeCalida
Archivo de tweets ya existe
mariaceg7
usuario privado
amilita9
Archivo de tweets ya 

agusBJok
Archivo de tweets ya existe
Usuario_borrado
juanchi_2312
Archivo de tweets ya existe
fcortez530
Archivo de tweets ya existe
GabrielRamiz
Archivo de tweets ya existe
francoroldan_10
usuario privado
elizabetharamb
Archivo de tweets ya existe
Nahir__Celeste
Archivo de tweets ya existe
Reinaa_MICA
Archivo de tweets ya existe
ClariStyles_1D
Archivo de tweets ya existe
queenmicxela
Archivo de tweets ya existe
juanaa97446936
Archivo de tweets ya existe
Joohacarp
Archivo de tweets ya existe
supayabe16
Archivo de tweets ya existe
Carlos0520683
Gastontin
Archivo de tweets ya existe
AbbyReales_
Archivo de tweets ya existe
FharmonyMica
Archivo de tweets ya existe
jr7fer
Archivo de tweets ya existe
NoeliaGoonzalez
Archivo de tweets ya existe
micaela_misiste
Archivo de tweets ya existe
Ludmi63455234
Archivo de tweets ya existe
MicaCapitana3
Archivo de tweets ya existe
Ntcn_RSS
Archivo de tweets ya existe
CamilaC58172408
Archivo de tweets ya existe
Leandro03245581
Archivo de tweets ya existe

<class 'dict'>
547765185
924124446025232386
1097158578
2159904902
487766088
1030986935353962496
924287840959303681
802903737169838080
956945495951278080
565873001
974804783163994112
951956159451357186
924277875720646656
924320615422504960
924304730091937794
921187751160098816
801633350926630912
924115420411547648
876039941733613568
931229296668106752
772165142402306048
924320843001249792
933902701057986560
924290685670477824
702059821445599232
906584040769122304
924105903254724609
442857983
280252550
893333790361366528
924223698546159616
786650624884637696
305794296
2440699669
924265345493360641
3631967966
924302429637824512
924125048188887040
535877868
921653625285431296
314194141
755482382497611777
924205811760590848
924114112992497665
791815161837420544
924297280319930368
883375134685491200
924135414054772736
619540832
468808898
904995307250966532
924108025400569856
924300747151470595
924118308726636545
924135392466661377
353159840
721431886556446721
118750326
2669274994
106904342
7